In [1]:
from flask import Flask, jsonify, request
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage

In [ ]:
from flask import Flask, jsonify, request
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from flask_cors import CORS, cross_origin


# Initialize the OpenAI API key and LLM model
OPENAI_API_KEY = ' '
llm = ChatOpenAI(temperature=1, openai_api_key=OPENAI_API_KEY)

# Database URI and setup
host = '127.0.0.1'
port = '3306'
username = 'root'
password = ''
database_schema = 'dasari'
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"

db = SQLDatabase.from_uri(mysql_uri, include_tables=['data_'], sample_rows_in_table_info=13)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

# Initialize the Flask application
app = Flask(__name__)
cors = CORS(app, resources={r"/api/*": {"origins": "http://192.168.1.98:3000"}})

def retrieve_from_db(query: str) -> str:
    try:
        db_context = db_chain(query)
        db_context = db_context['result'].strip()
        print(f"Debug: SQL Query Result - {db_context}")
        return db_context
    except Exception as e:
        print(f"Error: {e}")
        return "An error occurred while retrieving data from the database."

def generate(query: str) -> str:
    db_context = retrieve_from_db(query)
    
    system_message = """You are a professional representative of a financial institution.
    You have to answer user's queries and provide relevant information to help them understand their loan eligibility. 
    Example:
    
    Input:
    What are the criteria for getting a home loan?
    
    Context:
    The criteria for getting a home loan are:
    1. A minimum credit score of 650.
    2. A stable income for at least 2 years.
    3. A debt-to-income ratio below 40%.
    
    Output:
    The criteria for getting a home loan include having a minimum credit score of 650, a stable income for at least 2 years, and a debt-to-income ratio below 40%.
    """
    
    human_qry_template = HumanMessagePromptTemplate.from_template(
        """Input:
        {human_input}
        
        Context:
        {db_context}
        
        Output:
        """
    )
    messages = [
        SystemMessage(content=system_message),
        human_qry_template.format(human_input=query, db_context=db_context)
    ]
    try:
        response = llm(messages).content
        print(f"Debug: LLM Response - {response}")
        return response
    except Exception as e:
        print(f"Error: {e}")
        return "An error occurred while generating the response."

@app.route('/', methods=['GET'])
def home():
    return "Hello World!"

@app.route('/api/query/', methods=['GET', 'POST'])
@cross_origin()
def get_query_response():
    query_input = request.args.get('q')
    if not query_input:
        return jsonify({"error": "Query parameter 'q' is required."}), 400

    if query_input.lower() in ['hi', 'hello', 'hey']:
        return jsonify({"response": "Hello! How can I assist you with information about our financial institution?"})

    response_data = generate(query_input)
    return jsonify({"response": response_data})

if __name__ == '__main__':
    app.run(debug=False, host='127.0.0.1', port=5000)


C:\Users\NISHITHA SAI\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Jun/2024 22:02:55] "GET /api/query/?q=hi HTTP/1.1" 200 -
C:\Users\NISHITHA SAI\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
Give top 10 loan eligibility people list in total databsae
SQLQuery:SELECT `Id`, `Income`, `Age`, `Experience`, `Married/Single`, `House_Ownership`, `Car_Ownership`, `Profession`, `CITY`, `STATE`, `CURRENT_JOB_YRS`, `CURRENT_HOUSE_YRS`, `Risk_Flag`
FROM data_
ORDER BY `Income` DESC
LIMIT 10;
SQLResult: [(48740, 9999180, 22, 0, 'single', 'rented', 'no', 'Drafter', 'Bhavnagar', 'Gujarat', 0, 11, 0), (16395, 9999180, 22, 0, 'single', 'rented', 'no', 'Drafter', 'Bhavnagar', 'Gujarat', 0, 11, 0), (64996, 9999180, 22, 0, 'single', 'rented', 'no', 'Drafter', 'Bhavnagar', 'Gujarat', 0, 11, 0), (37937, 9999180, 22, 0, 'single', 'rented', 'no', 'Drafter', 'Bhavnagar', 'Gujarat', 0, 11, 0), (232, 9999180, 22, 0, 'single', 'rented', 'no', 'Drafter', 'Bhavnagar', 'Gujarat', 0, 11, 0), (54165, 9999180, 22, 0, 'single', 'rented', 'no', 'Drafter', 'Bhavnagar', 'Gujarat', 0, 11, 0), (32545, 9999180, 22, 0, 'single', 'rented', 'no', 'Drafter', 'Bhavnagar', 'Guj

C:\Users\NISHITHA SAI\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
127.0.0.1 - - [19/Jun/2024 22:10:37] "GET /api/query/?q=Give+top+10+loan+eligibility+people+list+in+total+databsae HTTP/1.1" 200 -


Debug: LLM Response - The top 10 loan eligibility people in the database are as follows:
1. Id: 48740, Income: 9999180, Age: 22, Experience: 0, Marital Status: single, House Ownership: rented, Car Ownership: no, Profession: Drafter, City: Bhavnagar, State: Gujarat, Current Job Years: 0, Current House Years: 11, Risk Flag: 0
2. Id: 16395, Income: 9999180, Age: 22, Experience: 0, Marital Status: single, House Ownership: rented, Car Ownership: no, Profession: Drafter, City: Bhavnagar, State: Gujarat, Current Job Years: 0, Current House Years: 11, Risk Flag: 0
3. Id: 64996, Income: 9999180, Age: 22, Experience: 0, Marital Status: single, House Ownership: rented, Car Ownership: no, Profession: Drafter, City: Bhavnagar, State: Gujarat, Current Job Years: 0, Current House Years: 11, Risk Flag: 0
4. Id: 37937, Income: 9999180, Age: 22, Experience: 0, Marital Status: single, House Ownership: rented, Car Ownership: no, Profession: Drafter, City: Bhavnagar, State: Gujarat, Current Job Years: 0, C